In [ ]:
!gfortran -Wall -Wextra -Wimplicit-interface -fPIC -fmax-errors=1 -g -fcheck=all -fbacktrace sin_accuracy.f90 -o sin_accuracy
!gfortran -Wall -Wextra -Wimplicit-interface -fPIC -fmax-errors=1 -g -fcheck=all -fbacktrace sin_accuracy_pure.f90 -o sin_accuracy_pure
!gfortran -Wall -Wextra -Wimplicit-interface -fPIC -fmax-errors=1 -g -fcheck=all -fbacktrace sin_accuracy_pure_kernel.f90 -o sin_accuracy_pure_kernel
!./sin_accuracy > sin_data.txt
!./sin_accuracy_pure > sin_pure_data.txt
!./sin_accuracy_pure_kernel > sin_pure_data_kernel.txt

In [ ]:
%pylab inline
from flint import ctx, arb
ctx.pretty = True
ctx.unicode = True 
ctx.dps = 50

In [ ]:
def compute_sin_arb(x):
    y = empty(size(x), dtype=arb)
    for i in range(size(x)):
        y[i] = arb(x[i]).sin()
    return y

In [ ]:
def floor(x):
    return int(x)

def mymod(x):
    y = empty(size(x), dtype="double")
    y2 = 2*pi
    for i in range(size(x)):
        y[i] = x[i] - floor(x[i]/y2)*y2
    return y

def arbmod(x):
    y = empty(size(x), dtype=arb)
    y2 = 2*arb.pi()
    for i in range(size(x)):
        y[i] = x[i] - (x[i]/y2).floor()*y2
    return y

In [ ]:
D = loadtxt("sin_data.txt")
x = D[:,0]
sin_gf = D[:,1]
sin_arb = compute_sin_arb(x)
D = loadtxt("sin_pure_data.txt")
assert(max(abs(x-D[:,0])) < 1e-16)
sin_pure = D[:,1]

err_gf = abs(sin_gf - sin_arb)
err_pure = abs(sin_pure - sin_arb)

mod_pure = mymod(x)
mod_arb = arbmod(x)
err_mod = abs(mod_pure-mod_arb)


figure(figsize=(12, 8))
loglog(x, err_gf, label="GFortran Intrinsic")
loglog(x, err_pure, label="Pure")
loglog(x, err_mod, "--", label="Modulo")
legend()
xlabel("x")
ylabel("Error of sin(x)")
#xlim([1, 1e5])
#ylim([1e-17, 1e-12])
grid()
show()

In [ ]:
D = loadtxt("sin_pure_data_kernel.txt")
x = D[:,0]
sin_pure_kernel = D[:,1]
sin_arb = compute_sin_arb(x)

err_pure_kernel = abs(sin_pure_kernel - sin_arb)


figure(figsize=(12, 8))
plot(x, err_pure_kernel, label="Pure kernel")
legend()
xlabel("x")
ylabel("Error of sin(x)")
grid()
show()